In [1]:
from yolov3 import YoloV3Net
import torch
import pandas

/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print('Loading network.....')
model = YoloV3Net('cfg/yolov3.cfg')
model.load_weights('weights/yolov3.weights')
model.net_info['height'] = '416'
model.eval()
CUDA = torch.cuda.is_available()
print('Network successfully loaded')

Loading network.....
Network successfully loaded


In [3]:
from util import load_classes
classes = load_classes('data/coco.names')

In [4]:
import pandas as pd
from PIL import ImageDraw, Image
import os
from util import load_classes
from metrics import get_annotation_bbox, draw_bbox


classes = load_classes('data/coco.names')
bbox_dict = dict()
dir = os.path.join('coco128', 'images', 'train2017')
for i,image_file in enumerate(os.listdir(dir)):
    image = Image.open(os.path.join(dir, image_file))
    bboxes = get_annotation_bbox(image_file[:-4], image.size, classes)
    if bboxes is not None:
        bbox_dict[image_file] = bboxes
        for bbox in bboxes:
            draw_bbox(image, bbox)

In [5]:
from metrics import generate_predictions

preds, target = generate_predictions(model, dir, classes, CUDA=CUDA)

In [6]:
from metrics import mean_average_precision

mean_average_precision(
    pred_boxes=preds,
    true_boxes=target,
    iou_thresh=0.5,
    box_format='corners',
    num_classes=len(classes)
)

tensor(0.5391)